In [40]:
# Import dependencies
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from joblib import dump

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd

# Don't need the following probably
import seaborn as sns 
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

import nltk

# Data cleaning and prep

In [37]:
# Read in CSV File
raw_data=pd.read_csv('db/winequality-red.csv', dtype={'Zip': 'str'})
raw_data.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5


In [5]:
# Investigate the dataset
raw_data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [7]:
# Shape of the data
raw_data.shape

(1599, 12)

In [8]:
# Check for NaN values
raw_data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [16]:
raw_data.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [9]:
raw_data["quality"].unique()

array([5, 6, 7, 4, 8, 3])

In [33]:
# Have to raplce lables from 3-8 to 0-5 
# NOTE: Had to replace the labels in two buckets, otherwise the data overlaps and gives an error
# TODO: Refactor the code to Map method 
first_replaced_data = raw_data.replace({'quality': {3: 0, 4: 1, 5: 2}})
final_replaced_data = first_replaced_data.replace({'quality': {6: 3, 7: 4, 8: 5}})

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2


In [36]:
# Now check to see if our labels are 0-5 
final_replaced_data["quality"].unique()

array([2, 3, 4, 1, 5, 0])

# Machine Learning 

In [38]:
# Data Preprocessing
X = final_replaced_data.drop("quality", axis=1)
y = final_replaced_data["quality"]
print(X.shape, y.shape)

(1599, 11) (1599,)


In [53]:
# Split the data into tarining and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [69]:
# Sacle the data (using MinMax) 
# TODO: See if I should rather use Standard Scaler instead
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
# Data normalization 
# X_scaler = StandardScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [57]:
# print(y_train)

In [70]:
# One-hot encoding
# TODO: Learn why we should perform one hot encoding in this case specifically
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Creating and defining Model Architecture (the layers)


In [71]:
# Create a model
model = Sequential()

In [72]:
from tensorflow.keras.layers import Dense
number_inputs = 11
number_hidden_nodes = 4
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [73]:
# Number of predicted classes and softmax
number_classes = 6
model.add(Dense(units=number_classes, activation='softmax'))

In [74]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4)                 48        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 30        
Total params: 78
Trainable params: 78
Non-trainable params: 0
_________________________________________________________________


In [75]:
# Compiling model
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [76]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
1199/1199 - 0s - loss: 1.7348 - acc: 0.2969
Epoch 2/500
1199/1199 - 0s - loss: 1.6709 - acc: 0.3945
Epoch 3/500
1199/1199 - 0s - loss: 1.5995 - acc: 0.4028
Epoch 4/500
1199/1199 - 0s - loss: 1.5163 - acc: 0.4362
Epoch 5/500
1199/1199 - 0s - loss: 1.4097 - acc: 0.5346
Epoch 6/500
1199/1199 - 0s - loss: 1.3006 - acc: 0.5263
Epoch 7/500
1199/1199 - 0s - loss: 1.2347 - acc: 0.5096
Epoch 8/500
1199/1199 - 0s - loss: 1.1896 - acc: 0.5338
Epoch 9/500
1199/1199 - 0s - loss: 1.1576 - acc: 0.5405
Epoch 10/500
1199/1199 - 0s - loss: 1.1349 - acc: 0.5413
Epoch 11/500
1199/1199 - 0s - loss: 1.1191 - acc: 0.5546
Epoch 12/500
1199/1199 - 0s - loss: 1.1071 - acc: 0.5463
Epoch 13/500
1199/1199 - 0s - loss: 1.0964 - acc: 0.5530
Epoch 14/500
1199/1199 - 0s - loss: 1.0877 - acc: 0.5546
Epoch 15/500
1199/1199 - 0s - loss: 1.0801 - acc: 0.5605
Epoch 16/500
1199/1199 - 0s - loss: 1.0737 - acc: 0.5613
Epoch 17/500
1199/1199 - 0s - loss: 1.0684 - acc: 0.5530
Epoch 18/500
1199/1199 - 0s - loss: 1.06

Epoch 145/500
1199/1199 - 0s - loss: 0.9467 - acc: 0.6047
Epoch 146/500
1199/1199 - 0s - loss: 0.9469 - acc: 0.6013
Epoch 147/500
1199/1199 - 0s - loss: 0.9464 - acc: 0.6047
Epoch 148/500
1199/1199 - 0s - loss: 0.9460 - acc: 0.6013
Epoch 149/500
1199/1199 - 0s - loss: 0.9457 - acc: 0.6063
Epoch 150/500
1199/1199 - 0s - loss: 0.9464 - acc: 0.6022
Epoch 151/500
1199/1199 - 0s - loss: 0.9454 - acc: 0.6047
Epoch 152/500
1199/1199 - 0s - loss: 0.9457 - acc: 0.6038
Epoch 153/500
1199/1199 - 0s - loss: 0.9450 - acc: 0.6063
Epoch 154/500
1199/1199 - 0s - loss: 0.9447 - acc: 0.6055
Epoch 155/500
1199/1199 - 0s - loss: 0.9450 - acc: 0.6063
Epoch 156/500
1199/1199 - 0s - loss: 0.9443 - acc: 0.6022
Epoch 157/500
1199/1199 - 0s - loss: 0.9450 - acc: 0.6047
Epoch 158/500
1199/1199 - 0s - loss: 0.9442 - acc: 0.6030
Epoch 159/500
1199/1199 - 0s - loss: 0.9435 - acc: 0.6047
Epoch 160/500
1199/1199 - 0s - loss: 0.9435 - acc: 0.6047
Epoch 161/500
1199/1199 - 0s - loss: 0.9438 - acc: 0.6055
Epoch 162/500


Epoch 287/500
1199/1199 - 0s - loss: 0.9314 - acc: 0.6113
Epoch 288/500
1199/1199 - 0s - loss: 0.9308 - acc: 0.6097
Epoch 289/500
1199/1199 - 0s - loss: 0.9309 - acc: 0.6097
Epoch 290/500
1199/1199 - 0s - loss: 0.9309 - acc: 0.6113
Epoch 291/500
1199/1199 - 0s - loss: 0.9316 - acc: 0.6072
Epoch 292/500
1199/1199 - 0s - loss: 0.9310 - acc: 0.6097
Epoch 293/500
1199/1199 - 0s - loss: 0.9307 - acc: 0.6022
Epoch 294/500
1199/1199 - 0s - loss: 0.9303 - acc: 0.6055
Epoch 295/500
1199/1199 - 0s - loss: 0.9300 - acc: 0.6063
Epoch 296/500
1199/1199 - 0s - loss: 0.9307 - acc: 0.6063
Epoch 297/500
1199/1199 - 0s - loss: 0.9300 - acc: 0.6030
Epoch 298/500
1199/1199 - 0s - loss: 0.9303 - acc: 0.6088
Epoch 299/500
1199/1199 - 0s - loss: 0.9304 - acc: 0.6088
Epoch 300/500
1199/1199 - 0s - loss: 0.9298 - acc: 0.6072
Epoch 301/500
1199/1199 - 0s - loss: 0.9298 - acc: 0.6097
Epoch 302/500
1199/1199 - 0s - loss: 0.9296 - acc: 0.6072
Epoch 303/500
1199/1199 - 0s - loss: 0.9296 - acc: 0.6088
Epoch 304/500


Epoch 429/500
1199/1199 - 0s - loss: 0.9225 - acc: 0.6088
Epoch 430/500
1199/1199 - 0s - loss: 0.9227 - acc: 0.6088
Epoch 431/500
1199/1199 - 0s - loss: 0.9223 - acc: 0.6122
Epoch 432/500
1199/1199 - 0s - loss: 0.9222 - acc: 0.6122
Epoch 433/500
1199/1199 - 0s - loss: 0.9224 - acc: 0.6122
Epoch 434/500
1199/1199 - 0s - loss: 0.9221 - acc: 0.6088
Epoch 435/500
1199/1199 - 0s - loss: 0.9222 - acc: 0.6097
Epoch 436/500
1199/1199 - 0s - loss: 0.9222 - acc: 0.6155
Epoch 437/500
1199/1199 - 0s - loss: 0.9220 - acc: 0.6138
Epoch 438/500
1199/1199 - 0s - loss: 0.9219 - acc: 0.6097
Epoch 439/500
1199/1199 - 0s - loss: 0.9223 - acc: 0.6180
Epoch 440/500
1199/1199 - 0s - loss: 0.9216 - acc: 0.6113
Epoch 441/500
1199/1199 - 0s - loss: 0.9214 - acc: 0.6130
Epoch 442/500
1199/1199 - 0s - loss: 0.9218 - acc: 0.6147
Epoch 443/500
1199/1199 - 0s - loss: 0.9215 - acc: 0.6122
Epoch 444/500
1199/1199 - 0s - loss: 0.9220 - acc: 0.6122
Epoch 445/500
1199/1199 - 0s - loss: 0.9213 - acc: 0.6113
Epoch 446/500


In [68]:
# Quantify the model
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

400/400 - 0s - loss: 0.9688 - acc: 0.5875
Loss: 0.9687782955169678, Accuracy: 0.5874999761581421
